<a href="https://colab.research.google.com/github/bachbech/job-offers/blob/main/job_offer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install spacy_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.8/190.8 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 33.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 71.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 70.2 MB/s eta 0:00:00


In [2]:
!pip install spacy

In [3]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json

In [4]:
spacy.__version__

'3.6.1'

In [5]:
!nvidia-smi

Tue Sep  5 17:03:36 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8     9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
cv_data=json.load(open('/content/data/training/train_data.json','r'))

In [7]:
len(cv_data)

22

In [9]:
!python -m spacy init fill-config /content/data/training/base_config.cfg /content/data/training/config.cfg

2023-09-05 17:07:33.970143: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Auto-filled config with all values
✔ Saved config
/content/data/training/config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [10]:
cv_data

[["Développement WEB FISH EYE TECHNOLOGIES · Paris, Île-de-France, France À propos de l’offre d’emploi Cette offre d’emploi provient d’un site d’offres d’emploi. En savoir plusFISH EYE TECHNOLOGIES, Cabinet de recrutement recrute un ingénieur de développement confirmé pour prendre en charge des projets de développement Web Vous aurez un rôle clé dans la conception, le développement et la maintenance de solutions Web avancées. Vous travaillerez en étroite collaboration avec l'équipe de développement et les parties prenantes pour garantir que les projets sont livrés avec succès, respectent les exigences et répondent aux normes de qualité. Responsabilités : Conception et développement Web : Concevoir, développer et mettre en œuvre des applications Web complexes, en utilisant les langages et les technologies appropriés. Assurer une architecture robuste, évolutive et sécurisée pour les applications Web. Maintenance et optimisation : Assurer la maintenance et les mises à jour régulières des 

In [11]:
def get_spacy_doc(file,data):
  nlp=spacy.blank('fr')
  db=DocBin()
  for text,annot in tqdm(data):
    doc=nlp.make_doc(text)
    annot=annot['entities']

    ents=[]
    entity_indices=[]
    for start,end,label in annot:
      skip_entity=False
      for idx in range(start,end):
        if idx in entity_indices:
          skip_entity=True
          break
      if skip_entity==True:
          continue
      entity_indices=entity_indices+list(range(start,end))
      try:
        span=doc.char_span(start,end,label=label,alignment_mode='contract')
      except:
        continue
      if span is None:
        err_data=str([start,end])+" "+str(text)+"\n"
        file.write(err_data)
      else:
        ents.append(span)
    try:
      doc.ents=ents
      db.add(doc)
    except:
      pass
  return db


In [12]:
from sklearn.model_selection import train_test_split
train,test=train_test_split(cv_data,test_size=0.1)

In [13]:
len(train),len(test)

(19, 3)

In [14]:
file=open('error.txt','w')
db=get_spacy_doc(file,train)
db.to_disk('train_data.spacy')

db=get_spacy_doc(file,test)
db.to_disk('test_data.spacy')

file.close()

100%|██████████| 3/3 [00:00<00:00, 55.58it/s]


In [15]:
!python -m spacy train /content/data/training/config.cfg --output ./output --paths.train ./train_data.spacy --paths.dev ./test_data.spacy --gpu-id 0


2023-09-05 17:11:08.243434: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
✔ Created output directory: output
ℹ Saving to output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were

In [16]:
nlp=spacy.load('/content/output/model-best')

In [17]:
doc=nlp("Développement WEB FISH EYE TECHNOLOGIES · Paris, Île-de-France, France À propos de l’offre d’emploi Cette offre d’emploi")
for ent in doc.ents:
  print(ent.text," == ",ent.label_)

Développement WEB FISH EYE  ==  titre


In [18]:
!pip install PyMuPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 29.4 MB/s eta 0:00:00


In [19]:
import sys,fitz

In [20]:
fname="/content/data/test/job offer2.pdf"
doc=fitz.open(fname)

In [21]:
doc=[page for page in doc]

In [22]:
try:
    doc = fitz.open(fname)
    text = " "
    for page in doc:
        text = text + str(page.get_text())
    doc.close()
except Exception as e:
    print(f"An error occurred: {e}")

In [23]:
text=text.strip()

In [24]:
text=text.replace("\n"," ")

In [25]:
text

"'Analyste développeur (H/F)  Eotim · Paris, Île-de-France, France (Hybride) À propos de l’offre d’emploi  Depuis 2001, Eotim est le leader du recrutement spécialisé en informatique et en marque  employeur. Nous accompagnons nos partenaires et nos clients dans leur croissance avec une  approche 100% sur-mesure. En parallèle et avec notre équipe de communication / marketing RH,  nous développons des solutions de gestion de la marque employeur. Présents en France, à Paris et  Caen, et à Lisbonne au Portugal, nous entretenons également des liens avec les États-Unis, le  Royaume-Uni et l''Australie pour accompagner nos clients internationaux. Nous cultivons un esprit  familial et une forte cohésion d''équipe au sein de notre entreprise. De ce fait, nous attachons  beaucoup d''importance à l''investissement humain, à l''humilité et veillons à effectuer un suivi  personnalisé et de qualité durant chacune de nos missions.      À PROPOS :      Réel accompagnateur des personnes en difficultés, 

In [26]:
doc=nlp(text)
for ent in doc.ents:
  print(ent.text," == ",ent.label_)

'Analyste développeur (H/F)  ==  titre
3  ==  experience
Symfony/  ==  skills


In [27]:
output_dir = '/content/saved_model'

# Load the trained model
nlp = spacy.load('/content/output/model-best')

# Save the model to the specified directory
nlp.to_disk(output_dir)